In [28]:
import pandas as pd
import numpy as np
import datetime # fechas y horas
import re # texto
import string # texto2
import matplotlib.pyplot as plt # graficar
import seaborn as sns # graficar2
import io

from sklearn.feature_extraction.text import CountVectorizer # NLP
from wordcloud import WordCloud
import emoji  

pd.set_option('display.max_rows', 500)

In [29]:
path='Chat de WhatsApp.txt'
df_whats = pd.read_csv(path, sep='delimiter', header=None, encoding='utf8', engine='python')

In [30]:
df_whats

,0
0,8/3/19 13:11 - Los mensajes en este grupo ahor...
1,"8/3/19 13:04 - x4 cre√≥ el grupo ""Bendita entre..."
2,8/3/19 13:11 - x4 te a√±adi√≥
3,8/3/19 13:14 - x1: XD
4,8/3/19 13:15 - x2: Quien es Kike y quien ea Lalo?
5,8/3/19 13:15 - x2: ü§î
6,8/3/19 13:16 - x3: <Multimedia omitido>
7,8/3/19 13:17 - x2: Va jeje
8,8/3/19 13:25 - x4: Naaa mms Jonathan
9,8/3/19 13:25 - x4: Jajajaja


In [31]:
df_whats.shape

(7198, 1)

In [32]:
date = []
time = []
persons = []
messages = []
errors = []
for row in range(len(df_whats)):
    whats = df_whats.iloc[row][0].split(' - ')
    try:
        messages.append(whats[1].split(':')[1])
        date.append(whats[0].split()[0])
        time.append(whats[0].split()[1])
        persons.append(whats[1].split(':')[0])
    except:
        errors.append(row)

In [33]:
df_whats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7198 entries, 0 to 7197
Data columns (total 1 columns):
0    7198 non-null object
dtypes: object(1)
memory usage: 56.3+ KB


In [34]:
print('Errores totales en la conversaci√≥n de whats: ', len(errors))

('Errores totales en la conversaci\xc3\xb3n de whats: ', 51)


In [35]:
dict_data = {'fecha':date, 'hora':time, 'persona':persons, 'mensaje':messages}
df = pd.DataFrame(dict_data)

In [36]:
df.head(6)

,fecha,hora,mensaje,persona
0,8/3/19,13:14,XD,x1
1,8/3/19,13:15,Quien es Kike y quien ea Lalo?,x2
2,8/3/19,13:15,ü§î,x2
3,8/3/19,13:16,<Multimedia omitido>,x3
4,8/3/19,13:17,Va jeje,x2
5,8/3/19,13:25,Naaa mms Jonathan,x4


In [37]:
df.tail()

,fecha,hora,mensaje,persona
7142,2/1/20,07:52,Nosotros vamos a AC,x8
7143,2/1/20,07:54,üò•,x2
7144,2/1/20,07:54,Jejejeje,x2
7145,2/1/20,08:07,üëÄ,x5
7146,2/1/20,09:23,ü§îü§îü§î,x2


In [38]:
df.shape

(7147, 4)

In [39]:
def limpia_fechas(fecha):
    fecha_lista = [int(element) for element in fecha.split('/')]
    return datetime.datetime(2000 + fecha_lista[2], fecha_lista[1], fecha_lista[0])

In [40]:
def limpia_horas(hora):
    hora_lista = [int(element) for element in hora.split(':')]
    return datetime.time(hora_lista[0], hora_lista[1])

In [41]:
def pintar(Serie, titulo = '', color = 'forestgreen', indice = 'fecha'):
    y = Serie.tolist()
    x = range(1, len(y) + 1)

    # Valores m√≠nimos y m√°ximos
    val_min = min(y)
    val_max = max(y)

    # Pintar los datos
    sns.set(style = 'darkgrid', font_scale = 1.4)
    plt.subplots(figsize = (21, 9))
    sns.lineplot(x = x, y = y, marker = 'o', color = color, markersize = 11, linewidth = 3)

    # Nombres
    plt.title(titulo, fontsize = 19, weight = 'bold')
    if indice == 'fecha':
        # Qu√© rango y dominio mostrar
        plt.ylim(val_min - 5, val_max + 5)
        plt.xlim(0, len(x) + 1)
        plt.xticks(x, [x.strftime('%D') for x in Serie.index.tolist()], fontsize = 14.5)
    else:
        # Qu√© rango y dominio mostrar
        plt.ylim(val_min - 1, val_max + 1)
        plt.xlim(0, len(x) + 1)
        plt.xticks(x, [x.strftime('%r') for x in Serie.index.tolist()], fontsize = 14.5)
    plt.xticks(rotation = 90)

    plt.show()

In [42]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?¬ø\]\%', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‚Äò‚Äô‚Äú‚Äù‚Ä¶¬´¬ª]', '', text)
    text = re.sub('\n', ' ', text)
    # ja equivale a jajajaja/ja/jajaj
    text = re.sub(r'(ja)[ja]*', 'jajaja', text)
    text = re.sub(r'(je)[je]*', 'jejeje', text)
    return text

In [43]:
def extract_emojis(str):
    return ''.join(c for c in str if c in emoji.UNICODE_EMOJI)

In [44]:
def extract_text(str):
    return ''.join(c for c in str if c not in emoji.UNICODE_EMOJI)

In [45]:
df['fecha'] = df['fecha'].apply(limpia_fechas)

df['hora'] = df['hora'].apply(limpia_horas)

In [46]:
df[['fecha', 'hora']].describe()

,fecha,hora
count,7147,7147
unique,222,1036
top,2019-11-01 00:00:00,13:06:00
freq,158,47
first,2019-03-08 00:00:00,NaN
last,2020-01-02 00:00:00,NaN


In [27]:
df.to_csv('tablac.csv',index=False, encoding='utf-8')

In [47]:
cont = 0
dict_time = {}
for hour in range(0, 24):
    for minute in range(0, 60):
        cont += 1
        dict_time[datetime.time(hour, minute)] = cont
        #print(cont, '\t', hour, ':', minute)

In [48]:
df['lab_minutes'] = df['hora'].map(dict_time).copy()

In [49]:
df.head()

,fecha,hora,mensaje,persona,lab_minutes
0,2019-03-08,13:14:00,XD,x1,795
1,2019-03-08,13:15:00,Quien es Kike y quien ea Lalo?,x2,796
2,2019-03-08,13:15:00,ü§î,x2,796
3,2019-03-08,13:16:00,<Multimedia omitido>,x3,797
4,2019-03-08,13:17:00,Va jeje,x2,798


In [50]:
df.to_csv('tablad.csv',index=False, encoding='utf-8')